In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mlp import Layer, MultilayerPerceptron, SquaredError, Relu, Linear
from sklearn.metrics import r2_score

# -------------------- Loading MPG Dataset -------------------- #
class MPGDataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path

    def load_data(self):
        columns = ["mpg", "cylinders", "displacement", "horsepower", "weight",
                   "acceleration", "model year", "origin", "car name"]
        df = pd.read_csv(self.dataset_path, sep='\s+', names=columns, na_values="?")
        df.dropna(inplace=True)
        df.drop(columns=["car name"], inplace=True, errors='ignore')
        df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
        df.dropna(inplace=True)

        y = df["mpg"].values.reshape(-1, 1)
        X = df.drop(columns=["mpg"]).values
        return X, y

# Load dataset
dataset_path = 'mpg-data/auto-mpg.data'
dataloader = MPGDataLoader(dataset_path)
X, y = dataloader.load_data()

# -------------------- Data Preprocessing -------------------- #
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
X_val = scaler_x.transform(X_val)
X_test = scaler_x.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# -------------------- Hyperparameter Tuning: Network Width -------------------- #
input_dim = X_train.shape[1]

width_configurations = [
    16,   # Narrow network
    32,   # Medium width
    64,   # Standard width
    128,  # Wide network
    256   # Extra-wide network
]

results = {}

for width in width_configurations:
    print(f"\nTraining model with {width} neurons per hidden layer...")

    layers = [
        Layer(input_dim, width, Relu(), dropout_rate=0.2),
        Layer(width, width, Relu(), dropout_rate=0.2),
        Layer(width, width, Relu(), dropout_rate=0.2),
        Layer(width, 1, Linear())
    ]

    mlp = MultilayerPerceptron(layers)
    train_losses, val_losses = mlp.train(
        X_train, y_train, X_val, y_val,
        SquaredError(), learning_rate=0.001, batch_size=32, epochs=150, optimizer='vanilla', momentum=0.7
    )

    y_pred = mlp.forward(X_test, training=False)

    # Ensure y_pred is reshaped correctly before inverse transformation
    y_pred = y_pred.reshape(-1, 1)
    y_pred = scaler_y.inverse_transform(y_pred)

    # Also inverse-transform y_test for accurate comparison
    y_test_original = scaler_y.inverse_transform(y_test)

    # Compute R² Score
    r2 = r2_score(y_test_original, y_pred)

    # Store results
    results[width] = round(r2 * 100, 2)  # Convert to percentage for better readability
    print(f"R² Score for {width} neurons: {results[width]}%")

# Identify Best Width Configuration
best_width = max(results, key=results.get)
best_r2 = results[best_width]

# -------------------- Summary of Results -------------------- #
print("\n===== Summary of Runs =====")
for width, r2 in results.items():
    print(f"R² value for {width} neurons: {r2}%")

print(f"\nBest Width Configuration: {best_width} neurons per layer with R² = {best_r2}%")


Training model with 16 neurons per hidden layer...
Epoch 1/150 - Train Loss: 1.1769, Val Loss: 0.8536
Epoch 2/150 - Train Loss: 0.9300, Val Loss: 0.9107
Epoch 3/150 - Train Loss: 0.8839, Val Loss: 0.8362
Epoch 4/150 - Train Loss: 0.8103, Val Loss: 0.7660
Epoch 5/150 - Train Loss: 0.7444, Val Loss: 0.6964
Epoch 6/150 - Train Loss: 0.7208, Val Loss: 0.9517
Epoch 7/150 - Train Loss: 0.6826, Val Loss: 0.5870
Epoch 8/150 - Train Loss: 0.6216, Val Loss: 0.7286
Epoch 9/150 - Train Loss: 0.6984, Val Loss: 0.5616
Epoch 10/150 - Train Loss: 0.6716, Val Loss: 0.6464
Epoch 11/150 - Train Loss: 0.6450, Val Loss: 0.5938
Epoch 12/150 - Train Loss: 0.6126, Val Loss: 0.7785
Epoch 13/150 - Train Loss: 0.6431, Val Loss: 0.5767
Epoch 14/150 - Train Loss: 0.6087, Val Loss: 0.6473
Epoch 15/150 - Train Loss: 0.5039, Val Loss: 0.4831
Epoch 16/150 - Train Loss: 0.6266, Val Loss: 0.6218
Epoch 17/150 - Train Loss: 0.6333, Val Loss: 0.5722
Epoch 18/150 - Train Loss: 0.5128, Val Loss: 0.6146
Epoch 19/150 - Train 